In [ ]:
last_table = []

In [ ]:
import csv
score = {'가성비':0,'as':0,'배송':0,'성능':0} #총점
f = open('LG전자 2019 울트라PC.csv','r',encoding='utf-8-sig')
f2 = open('LG전자 2019 울트라PC 수정.csv','w',encoding='utf-8-sig')
r = csv.reader(f)
w = csv.writer(f2)
keyword = []
new_file = []
for i in score:
    keyword.append(i)

number = 0
for i in r:
    each_score = {'가성비':0,'as':0,'배송':0,'성능':0} #개별리뷰별 최상위 키워드에 대한 점수
    empty = []
    empty.append(i[0])
    dic = {} #해당 리뷰 내 최상위 키워드의 인덱스 넣는부분
    order = [] #단일 리뷰 내에서 출현한 최상위키워드를 순서대로 넣음
    number = number + 1
    for j in keyword: #키워드 첫 index 찾는부분
        num = i[0].find(j)
        dic[j] = num
    for k in dic: #index가 -1이 아니면 order list에 넣음
        if dic[k] !=-1:
            order.append(k)
            
    for l in range(len(order)-1): #리뷰에 최상위 키워드가 출현한 순서로 list 재배열
        for m in range(l+1,len(order)):
            if dic[order[l]]>dic[order[m]]:
                temp = order[l]
                order[l] = order[m]
                order[m] = temp
    for n in range(len(order)): #찾은 최상위 키워드의 인덱스를 사용하여 다음 최상위 키워드 전까지 슬라이싱하고 Predict
        if n != len(order)-1:
            line = i[0][dic[order[n]]:dic[order[n+1]]]
            each_score[order[n]] = predict(line)
            score[order[n]] = score[order[n]] + predict(line)
        else:
            line = i[0][dic[order[n]]:]
            each_score[order[n]] = predict(line)
            score[order[n]] = score[order[n]] + predict(line)
    for o in each_score: 
        empty.append(each_score[o])
        
    new_file.append(empty)
    
for line in new_file:
    w.writerow(line)
    
f.close()
f2.close()

last_score = ['LG전자 2019 울트라PC']
for j in score:
    sco = score[j]/number
    last_score.append(sco)

last_table.append(last_score)

In [ ]:
final = open('last_table.csv','w',encoding='utf-8-sig')
w = csv.writer(final)

for i in last_table:
    w.writerow(i)
    
final.close()